# DDoS_ML_Sup-V1.6
log 1.6: CFM -> ds_preprocess.py

## Résumé de différentes étapes:


### A - Créer une Dataset 
##### * Analyser les information requise
##### * Analyser la quantiter de paquets moyennés nécéssaire dans chaque info
#### * Créer une pré-dataset
#### * Transformer la pré-dataset en dataset final

### B - Créer un réseau de neurones convenable
##### * Analyser le nombre d'entrée
##### * Entrainer les neurones
##### * Analyser les résultats
##### * Ajuster

### C - Analyser les résultat de l'IA




### A - 1 - Créer une Dataset
- Poids moyen des paquets
- Delta time moyen entre chaque paquets
- Pourcentage d'uttilisation de chaque protocole
- Nombre de paquets par seconde
- Pourcentage de fois la meme IP src/dst
- Pourcentage de fois le meme port src/dst

### A - 2 - Créer une Dataset
1er éssaie: Un éssai sur des groupes de 100 paquets
1.5: Essaie sur 3 comm

### A - 3 - Créer une Dataset

#### Note:
Pourquoi pas rajouter l'écart moyen entre la len moyenne ?
Pourquoi pas rajouter le timeflow de chaque ?

###### Pré-Dataset:
Timestamp | Delta time | Len paquets fwd | Len paquets bwd | Ip src | port src | Ip dst | Port dst | Protocole | Label


###### Dataset final:
   Delta Time | Len paquets Fwd| Len paquets Bwd | Paquet/s | % meme Ip src | % meme port src | % meme Ip dst | % meme port dst |  totalité protocole

###### Params:

In [99]:
w_epochs = 3
w_batch_size = 1 #to add

percentage_for_test_data = 0.5
reduction_facteur = 0
size_list = 20

path_ModelCheckpoint = "models_saved/model-{epoch:03d}-batche_size=20.h5"

###### Création et utilisation de: ds_create.py

In [100]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import datetime
from sklearn.utils import shuffle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten

### Importation de la dataset

In [101]:
path_to_benign_csv = "/run/media/Thytu/TOSHIBA EXT/PoC/Smartshark/DS/V2/Benigns/ds_flow1_900k_batch_size_20.csv"
path_to_ddos_csv = "/run/media/Thytu/TOSHIBA EXT/PoC/Smartshark/DS/V2/DDoS/ds_ddos_900k_D1_batch_size_20.csv"

### Avoir une idée de la dataset

In [102]:
benign_csv = pd.read_csv(path_to_benign_csv, names=["Delta time", "Len", "Protocol", "Total Delta", "Total Len", "Average Delta", "Average Len", "Delta Std", "Len Std"], dtype='float')
ddos_csv = pd.read_csv(path_to_ddos_csv, names=["Delta time", "Len", "Protocol", "Total Delta", "Total Len", "Average Delta", "Average Len", "Delta Std", "Len Std"], dtype='float')

In [103]:
if len(benign_csv) > len(ddos_csv):
    benign_csv = benign_csv[:len(ddos_csv)]
else:
    ddos_csv = ddos_csv[:len(benign_csv)]

In [104]:
benign_csv.tail()

,Delta time,Len,Protocol,Total Delta,Total Len,Average Delta,Average Len,Delta Std,Len Std
899995,0.000312,320.0,17.0,0.012139,4447.0,1.348785e-08,0.004941,0.000865,264.906893
899996,0.001942,90.0,17.0,0.014081,4537.0,1.564563e-08,0.005041,0.000845,256.791136
899997,0.004135,72.0,17.0,0.018216,4609.0,2.024007e-08,0.005121,0.000866,252.787778
899998,0.000279,324.0,17.0,0.018495,4933.0,2.055005e-08,0.005481,0.001133,249.472109
899999,0.000158,100.0,17.0,0.018653,5033.0,2.072558e-08,0.005592,0.001115,243.364847


In [105]:
ddos_csv.tail()

,Delta time,Len,Protocol,Total Delta,Total Len,Average Delta,Average Len,Delta Std,Len Std
899995,0.000001,271.0,17.0,0.000640,4300.0,7.111151e-10,0.004778,0.000089,67.233788
899996,0.000307,271.0,17.0,0.000947,4571.0,1.052227e-09,0.005079,0.000087,65.287076
899997,0.000002,271.0,17.0,0.000949,4842.0,1.054448e-09,0.005380,0.000105,63.498467
899998,0.000064,271.0,17.0,0.001013,5113.0,1.125558e-09,0.005681,0.000103,61.847907
899999,0.000001,271.0,17.0,0.001014,5384.0,1.126668e-09,0.005982,0.000100,60.318683


### Préparation finale de la dataset
#### Mélange des deux dataset pour en former plus qu'une seul et création des labels

#### Création de la DS de test

In [106]:
def print_test_results(dataset, label, test_dataset, label_test):
    x = True if len(dataset) == len(label) else False
    print(f"Is {len(dataset)} equale to {len(label)} ?  {x}")
    x = True if len(test_dataset) == len(label_test) else False
    print(f"Is {len(test_dataset)} equale to {len(label_test)} ?  {x}")

def Creating_splited_datasets(benign_csv, ddos_csv, percentage_for_test_data, reduction_facteur=0):
    benign_csv = benign_csv.values.tolist()
    ddos_csv = ddos_csv.values.tolist()
    
    dataset = benign_csv[:int(len(benign_csv) * (1 - reduction_facteur))]
    test_dataset = dataset[int(len(dataset) * (1 - percentage_for_test_data)):]
    label = [0] * int(len(dataset) * (1 - percentage_for_test_data))
    label_test = [0] * len(dataset[int(len(dataset) * (1 - percentage_for_test_data)):])
    dataset = dataset[:int(len(dataset) * (1 - percentage_for_test_data))]

    print_test_results(dataset, label, test_dataset, label_test)

    tmp = ddos_csv[:int(len(ddos_csv) * (1 - reduction_facteur))]
    tmp_test = tmp[int(len(tmp) * (1 - percentage_for_test_data)):]
    label += [1] * int(len(tmp) * (1 - percentage_for_test_data))
    label_test += [1] * int(len(tmp_test))
    tmp = tmp[:int(len(tmp) * (1 - percentage_for_test_data))]
    
    dataset += tmp
    test_dataset += tmp_test
    
    print_test_results(dataset, label, test_dataset, label_test)
    
    return dataset, label, test_dataset, label_test

In [107]:
dataset, label, test_dataset, test_label = Creating_splited_datasets(benign_csv, ddos_csv, percentage_for_test_data, reduction_facteur)

print(label[0], label[len(label) - 1])
print(len(label), len(dataset))
print(len(test_label), len(test_dataset))

Is 450000 equale to 450000 ?  True
Is 450000 equale to 450000 ?  True
Is 900000 equale to 900000 ?  True
Is 900000 equale to 900000 ?  True
0 1
900000 900000
900000 900000


#### Diminiuer l'effet des grands nombres pour facilité l'apprentissage du réseau

In [108]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
dataset = scaler.fit_transform(dataset)
test_dataset = scaler.fit_transform(test_dataset)

#### Transformation de la list en np.array

In [109]:
label = np.array(label)
test_label = np.array(test_label)

print(label); print(test_label)
print(len(label)); print(len(dataset))
print(len(test_label)); print(len(test_dataset))

[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
900000
900000
900000
900000


#### Mise en séries

In [110]:
def listing_ds(dataset, label, size_list=20):
    result = []
    tmp = []
    r_label = []
    r = 0
    for index, data in enumerate(dataset):
        tmp.append(data)
        r = r + 1 if label[index] == 0 else r - 1
        if index % size_list == size_list - 1:
            result.append(np.array(tmp))
            if r > 0:
                r_label.append(0)
            else:
                r_label.append(1)
            tmp = []
            r = 0
    return np.array(result), np.array(r_label)

In [111]:
print(len(label))
print(len(test_label))
dataset, label = listing_ds(dataset, label)
test_dataset, test_label = listing_ds(test_dataset, test_label, size_list)

print(dataset.shape, len(label))
print(test_dataset.shape, len(test_label))

900000
900000
(45000, 20, 9) 45000
(45000, 20, 9) 45000


#### Création du model 

In [112]:
model = Sequential()

model.add(LSTM(9, input_shape=dataset[0].shape, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(9, input_shape=dataset[0].shape, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(Dense(9, activation='relu'))
model.add(Flatten())

model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)

model.compile(loss='sparse_categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

##### Callbacks creation

In [113]:
logs_path = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_path, histogram_freq=1)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    path_ModelCheckpoint, monitor='val_acc', mode='max', save_best_only=False, verbose=0, #ATTENTION !!
    save_weights_only=False, save_freq='epoch')


#### Shuffle

In [114]:
#dataset, label = shuffle(dataset, label)

In [115]:
print(len(dataset), len(label))
print(dataset.shape)

45000 45000
(45000, 20, 9)


#### Entrainement du model

In [116]:
history = model.fit(dataset, label, epochs=w_epochs, batch_size=1, validation_data=(test_dataset, test_label), callbacks=[tensorboard_callback, checkpoint_callback])

Epoch 1/3
45000/45000 [==============================] - 353s 8ms/step - loss: 0.4924 - accuracy: 0.7610 - val_loss: 0.2073 - val_accuracy: 0.9100
Epoch 2/3
45000/45000 [==============================] - 362s 8ms/step - loss: 0.0581 - accuracy: 0.9915 - val_loss: 2.6546 - val_accuracy: 0.7944
Epoch 3/3
 1113/45000 [..............................] - ETA: 4:47 - loss: 0.0397 - accuracy: 0.9928

KeyboardInterrupt: 

#### Résultat sous graphique du model

In [ ]:
loss_curve = history.history["loss"]
acc_curve = history.history["accuracy"]

plt.plot(loss_curve)
plt.title("Loss")
plt.show()

plt.plot(acc_curve)
plt.title("Accuracy")
plt.show()

## Analyzing results

##### Creation of data to evaluate false positives number on all data

In [117]:
false_positives_test = scaler.fit_transform(benign_csv.values.tolist())
#false_positives_test = scaler.fit_transform(benign_csv[0:-1])
#false_positives_label = np.array([0] * len(false_positives_test))
false_positives_label = [0] * len(false_positives_test)

false_positives_test, false_positives_label = listing_ds(false_positives_test, false_positives_label, size_list)

#### Evaluation of false positives number

In [118]:
model.evaluate(false_positives_test[0:1000], false_positives_label[0:1000], batch_size=1)

1000/1000 [==============================] - 2s 2ms/step - loss: 231.6558 - accuracy: 0.1480


[231.6558380126953, 0.14800000190734863]

In [119]:
aze = benign_csv.values.tolist()
print(type(aze))
aze = scaler.fit_transform(aze)
print(type(aze))
qwe = [0] * len(aze)
aze, qwe = listing_ds(aze, qwe, size_list)
model.evaluate(aze[0:1000], qwe[0:1000], batch_size=1)

<class 'list'>
<class 'numpy.ndarray'>
1000/1000 [==============================] - 3s 3ms/step - loss: 231.6558 - accuracy: 0.1480


[231.6558380126953, 0.14800000190734863]

In [120]:
def stest(dataset, labels):
    r = []
    rl = []
    for index, data in enumerate(dataset):
        if labels[index] == 0:
            r.append(data)
            rl.append(0)
    return np.array(r), np.array(rl)

In [121]:
r, rl = stest(dataset, label)
tr, trl = stest(test_dataset, test_label)
tt = np.concatenate((r, tr))
ttl = np.concatenate((rl, trl))

print(r[0][0][0]);print(false_positives_test[0][0][0]);print(type(r));print(type(false_positives_test))
model.evaluate(tt[0:1000], ttl[0:1000], batch_size=1)

-0.07937393942999518
-0.6212217280407947
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0082 - accuracy: 1.0000


[0.008166040293872356, 1.0]

In [ ]:
print(len(false_positives_test))
def know_perc_of_each(labels):
    b = 0
    d = 0
    for label in labels:
        if label == 0:
            b += 1
        else:
            d += 1
    print(f"There is {int(b * 100 / len(labels))}% of benign and {int(d * 100 / len(labels))}% of ddos")
know_perc_of_each(label)
know_perc_of_each(test_label)

##### Creation of data to evaluate false negatives number on all data

In [ ]:
#false_negatives_test = scaler.fit_transform(ddos_csv[len(ddos_csv) - 1_000_000:])
false_negatives_test = scaler.fit_transform(ddos_csv[0:-1])
false_negatives_label = np.array([1] * len(false_negatives_test))
#false_negatives_label = np.array([1] * 1_000_000)

false_negatives_test, false_negatives_label = listing_ds(false_negatives_test, false_negatives_label, size_list)

#### Evaluation of false negatives number

In [ ]:
model.evaluate(false_negatives_test, false_negatives_label, batch_size=1)

In [ ]:
print(false_positives_test.shape)
print(false_negatives_test.shape)
print(dataset.shape)

In [ ]:
for epoch in range(w_epochs):
    if epoch + 1 < 10:
        file_path = f"models_saved/model-00{epoch + 1}-batche_size=20.h5"
    else:
        file_path = f"models_saved/model-0{epoch + 1}-batche_size=20.h5"
    tmp_model = tf.keras.models.load_model(file_path)
    print("Epoch:", epoch + 1)
    print("False Positives: ")
    tmp_model.evaluate(false_positives_test, false_positives_label, batch_size=1)
    print("False Negatives: ")
    tmp_model.evaluate(false_negatives_test, false_negatives_label, batch_size=1)

In [ ]:
model.fit(test_dataset, test_label, epochs=30)

In [ ]:
model.evaluate(false_negatives_test[:1000], false_negatives_label[0:1000], batch_size=1)

In [124]:
tmp_model = tf.keras.models.load_model("models_saved/model-001-batche_size=20.h5")
tmp_model.evaluate(test_dataset, test_label, batch_size=1)

45000/45000 [==============================] - 104s 2ms/step - loss: 0.2073 - accuracy: 0.9100


[0.2072926014661789, 0.9099555611610413]